## Transform eta maps to javascript usable format
This notebook reads in h/eta values from h_eta_small.npy files and aggregates them as javascript variables in a single file.

### First import some necessary tools


In [1]:
%pylab inline
from __future__ import print_function
from ptha_paths import data_dir, events_dir, js_data_dir
import sys, os

Populating the interactive namespace from numpy and matplotlib
Assuming that top level of this repository is at: /media/brian/TBdrive/Documents/Coursework/CSE512/Homework/project/tsunami-inundation/ptha_tutorial
    Python codes can be found in codes_dir = /media/brian/TBdrive/Documents/Coursework/CSE512/Homework/project/tsunami-inundation/ptha_tutorial/PythonCode
    Data files can be found in data_dir = /media/brian/TBdrive/Documents/Coursework/CSE512/Homework/project/tsunami-inundation/ptha_tutorial/DataFiles
    Results for each event can be found in events_dir = /media/brian/TBdrive/Documents/Coursework/CSE512/Homework/project/tsunami-inundation/ptha_tutorial/DataFiles/Events
Assuming that top level of the CSE512 repository is at: /media/brian/TBdrive/Documents/Coursework/CSE512/Homework/project/tsunami-inundation
    JavaScript data files can be found in data_dir = /media/brian/TBdrive/Documents/Coursework/CSE512/Homework/project/tsunami-inundation/docs/data


### Set parameters 

In [26]:
# A dictionary of sets of events to consider
meta_events         = {}
meta_events['all']  = ['AASZa', 'AASZb', 'AASZc', 'AASZd', 'CSZa', 'CSZb', 'CSZc', 'CSZd', 'CSZe', \
                       'CSZf', 'KmSZa', 'KrSZa', 'SChSZa', 'TOHa']
meta_events['near'] = ['CSZa', 'CSZb', 'CSZc', 'CSZd', 'CSZe', 'CSZf']
meta_events['far']  = ['AASZa', 'AASZb', 'AASZc', 'AASZd', 'KmSZa', 'KrSZa', 'SChSZa', 'TOHa']

# Resolution of the data
nx = 250
ny = 250

# Name of file where we will output results
out_filename = 'simulation_data.js'

# Names of some javascript variables we wish to output
near_events = "near_events"
far_events  = "far_events"
all_events  = "all_events"

### Set the full names (or decodes) of the abbreviated event names

In [27]:
decodes = {}
decodes['AASZa']  = 'Alaska A'
decodes['AASZb']  = 'Alaska B'
decodes['AASZc']  = 'Alaska C'
decodes['AASZd']  = 'Alaska D'
decodes['CSZa']   = 'Cascadia A'
decodes['CSZb']   = 'Cascadia B'
decodes['CSZc']   = 'Cascadia C'
decodes['CSZd']   = 'Cascadia D'
decodes['CSZe']   = 'Cascadia E'
decodes['CSZf']   = 'Cascadia F'
decodes['KmSZa']  = 'Kamchatka'
decodes['KrSZa']  = 'Kurils'
decodes['SChSZa'] = 'Southern Chile'
decodes['TOHa']   = 'Tohoku'

### Set the annual probability for each event
We create a dictionary storing the probabilities of each event. These were given to us from Randy. We did not come up with them ourselves.

In [28]:
event_prob           = {}
event_prob['AASZa']  = 1./394.
event_prob['AASZb']  = 1./750.
event_prob['AASZc']  = 1./563.
event_prob['AASZd']  = 1./324.
event_prob['CSZa']   = 1./250. * .0125
event_prob['CSZb']   = 1./250. * .0125
event_prob['CSZc']   = 1./250. * .0750
event_prob['CSZd']   = 1./250. * .5000
event_prob['CSZe']   = 1./250. * .1750
event_prob['CSZf']   = 1./250. * .2250
event_prob['KmSZa']  = 1./50.
event_prob['KrSZa']  = 1./167.
event_prob['SChSZa'] = 1./300.
event_prob['TOHa']   = 1./103.

### Define a function for outputting lists in a javascript-friendly format

The main thing this function accomplishes is writing the entries of a list `out_list` to a file object `file_obj` with commas separating each item. For example, `l=[0,1,2,3,4,5]` would be written as `0,1,2,3,4,5`.

In [29]:
def write_list(file_obj, out_list):
    first_flag = True
    for entry in out_list:
        if first_flag:
            file_obj.write(str(entry))
            first_flag = False
        else:
            file_obj.write(',' + str(entry))

### Define a function for outputting javascript array variables

This function writes a python array to a file as a javascript array variable. For example, `write_js_array_variable(f,[0,1,2,3],"seq")` adds the following line to the file object `f`: `const seq = [0,1,2,3]`. That is, it defines a javascript array called `seq` containing the values of the second argument.

In [30]:
def write_js_array_variable(file_obj, out_list, var_name):
    file_obj.write("var %s = [" %(var_name))
    write_list(file_obj,out_list)
    file_obj.write("];\n")

### Loop over all events and output zeta to a javascript file

In [25]:
# Get location of file where we'll output the results
out_file = os.path.join(js_data_dir,out_filename)

first_event = True
for event in meta_events['all']:
    
    # Load the eta data file
    event_dir = os.path.join(events_dir, event)
    hmax_file = os.path.join(event_dir, 'h_eta_small.npy')
    hmax      = load(hmax_file)
    Hmax      = hmax.reshape((nx,ny),order='F')

    # Define some strings to help us format the output javascript-friendly way
    var_name    = "%s" %(event)
    name_pair   = "\"name\":\"%s\"" %(event)
    fname_pair  = "\"full_name\":\"%s\"" %(decodes[event])
    prob_pair   = "\"probability\":%f" %(event_prob[event])
#     label_pair  = "\"label\":\"%s (p=%f)\"" %(event, event_prob[event])
    decode_pair  = "\"decode\":\"{} (p={:.4f})\"".format(decodes[event], event_prob[event])
    
    
    
    header  = "var %s = {%s,%s,%s,%s,\"eta_data\":{\"width\":%d,\"height\":%d,\"values\":[" \
                 %(var_name, name_pair, fname_pair, prob_pair, decode_pair, nx, ny)
    footer  = "]}};\n"
    
    # Write the text to a file
    # We want the first event to overwrite the previous file
    if first_event:
        with open(out_file,'w') as f:
            f.write(header)
            write_list(f,Hmax.ravel())
            f.write(footer)
        first_event = False
    # Otherwise we just append incoming data as new rows in the same file
    else:
        with open(out_file,'a') as f:
            f.write(header)
            write_list(f,Hmax.ravel())
            f.write(footer)
        

# Write a few more variables to the file containing the variable/event names
with open(out_file,'a') as f:
    # All the events
    write_js_array_variable(f,meta_events['all'],all_events)
    
    # Near events
    write_js_array_variable(f,meta_events['near'],near_events)
    
    # Far events
    write_js_array_variable(f,meta_events['far'],far_events)